In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Saves local subgraphs from the larger traffic graph for hops 1...10

In [2]:
import os, sys
sys.path.append('..')
from glob import glob
from configs import *
from tqdm import tqdm_notebook as tqdm
# from tqdm.auto import tqdm
# tqdm.monitor_interval = 0
from utils import *
import numpy as np
from time import time
import json
import numpy as np
from time import time, strptime, mktime
import matplotlib.pyplot as plt
import cv2
from time import sleep

In [3]:
with open('../data/man.json') as fl:
    vs, adjlist = json.load(fl)

In [4]:
print(len(vs))
print(len(adjlist))

684
684


In [5]:
def subgraph(selected, vs, adj):
    subadj = {}
    for vert in selected:
        subadj[vert] = [v for v in adj[vert] if v in selected]
        
    return selected, subadj

def find_hops(hops, vs, adj):
    graphs = []
    cvs, cadj = complete_graph(vs, adj)
    
    for ri, root in enumerate(cvs):
        # search h-hops for each root
        visited = {}
        queue = [root]
        depth = {}
        depth[root] = 0
        exceeded = False
        
        maxd = 0
        while len(queue):
            head = queue.pop(0)
            visited[head] = True
            if depth[head] > maxd:
                maxd = depth[head]
            
            if depth[head] < hops:
                has_children = False
                
                for child in cadj[head]:
                    if child not in visited:
                        has_children = True
                        
                        queue.append(child)
                        depth[child] = depth[head] + 1
#                 if not has_children:
#                     exceeded = True
#                     break
        
#         if not exceeded:
        if maxd == hops:
            # if reached n-hops
            verts = [root]
            for vert in visited.keys(): 
                if vert not in verts: verts.append(vert)
            graphs.append((
                subgraph(verts, vs, adj), 
                depth
            ))
    
    
    return graphs
    
# for hops in range(1, 3):
for hops in range(1, 11):
    with open('../data/man.json') as fl:
        vs, adjlist = json.load(fl)
    h_graphs = find_hops(hops, vs, adjlist)
    
    rvs = len(h_graphs) / len(vs)
    
    print(hops, '%.1f%%' % (rvs * 100))
    sleep(1)
    
    if not os.path.isdir('images/hop_%d' % hops):
        os.mkdir('images/hop_%d' % hops)
    
    for ((iv, ia), depth) in tqdm(h_graphs):
        dlabel = lambda ent: '%d:%s' % (depth[ent], ent[0] + ent[-2:])
        gobj = show_graph(iv, ia, vdesc=dlabel)
        imname = 'images/hop_%d/%s_%d' % (hops, iv[0], hops)
        gobj.render(imname)
        
#         if iv[0] == '400950-400951':
        with open('../data/graphs/%s_n%d.json' % (iv[0], hops), 'w') as fl:
            json.dump([
                iv,
                ia
            ], fl, indent=4)
            
        
#         if iv[0] == '400258-400259':
#             gobj.render('images/samples/sample_%d' % hops)
        

1 100.0%


100%|██████████| 684/684 [00:11<00:00, 58.91it/s]


2 99.6%


100%|██████████| 681/681 [00:12<00:00, 56.27it/s]


3 98.5%


...

100%|██████████| 645/645 [00:14<00:00, 45.71it/s]


9 92.5%


100%|██████████| 633/633 [00:14<00:00, 43.78it/s]


10 90.9%


100%|██████████| 622/622 [00:14<00:00, 42.65it/s]


In [6]:
# with open('../data/graphs/400950-400951_n5.json') as fl:
#     lvs, ladjs = json.load(fl)
# print(len(lvs))

# with open('../data/manh_stops.json', 'w') as fl:
#     json.dump(lvs, fl, indent=4)

15


In [12]:
all_segs = {}
for fname in glob('../data/graphs/*.json'):
    with open(fname) as fl:
        lvs, _ = json.load(fl)
    for vert in lvs:
        all_segs[vert] = True
        
print(len(all_segs))

with open('../data/manh_stops.json', 'w') as fl:
    json.dump(list(all_segs.keys()), fl, indent=4)

684
